In [3]:
# Install required library (uncomment the line below if not already installed)
# !pip install inference-sdk

# Import necessary libraries
from inference_sdk import InferenceHTTPClient
import cv2
import numpy as np

# Set up the Inference Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="D6NnigvMvhlT8B4yVYU4"
)

# Define model ID
model_id = "offline-exam-monitoring-3-rub5m-y5zbj/1"

# Initialize webcam
cap = cv2.VideoCapture(0)

# Check if webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Convert the frame to RGB (as the inference model expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run inference on the current frame
    result = CLIENT.infer(rgb_frame, model_id=model_id)

    # Parse response for visualization
    detections = result['predictions']

    # Draw bounding boxes and labels on the frame
    for detection in detections:
        x, y, width, height = detection['x'], detection['y'], detection['width'], detection['height']
        x1 = int(x - width / 2)
        y1 = int(y - height / 2)
        x2 = int(x + width / 2)
        y2 = int(y + height / 2)

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Add label with class and confidence
        label = f"{detection['class']} ({detection['confidence']:.2f})"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Webcam Monitoring', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()
